In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain python-dotenv bs4

In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

True

In [5]:
os.environ["USER_AGENT"] = "Test App/1.0"

In [37]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import bs4

In [11]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

In [ ]:
print(docs)

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [17]:
splits = text_splitter.split_documents(docs)
print(len(splits))

66


In [23]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [24]:
retriever = vectorstore.as_retriever()

In [26]:
prompt = hub.pull("rlm/rag-prompt")

In [40]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [34]:
#post processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [44]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What is Task")